# Trie Project

Note: Documentation and code are both included within this notebook

In [1]:
# import
import pandas as pd

In [2]:
class TrieNode:

    def __init__(self, myCharName):
        # A trie node may always be associated with a character, except the Root Node
        #initialized TrieNode class with charName and 3 other attributes
        self.endToken = False  # True only when the node represents the end of the word
        self.ChildNodes = {}  # dict with chars as keys and nodes as their respective values, wasn't able to work with lists
        # this dict may only contain one char and node per run. Its node value contains its children
        self.char = myCharName # setting the name character of this node
        self.frequency = 0  # cumulative frequency initialized 

    def getNodeChar(self):
        #This method simply returns its name character
        return self.char

    def getFrequency(self, substring):
        #substring is a sliced or complete word that returns an int which is the substring's frequency
        for char in substring:
            if not self.getChildNode(char):
                break
            self = self.getChildNode(char)
        return self.frequency

    def getChildNode(self, childChar):
        # searches through ChildNodes, if node with character 'childChar' is found then returns childNode
        for i in self.ChildNodes:
            if i == childChar:
                return self.ChildNodes[childChar]
        return None

    def addEntry(self, substring, freq):
        # Loop through each character in the word
        # Check if there is no child containing the character, create a new child for the current node and store in ChildNode
        # endToken will be True since we reach last character of the substring
        for char in substring:
            if self.getChildNode(char) == None:
                self.ChildNodes[char] = TrieNode(char)
            self = self.getChildNode(char)
            self.frequency += freq
        self.endToken = True
#         print(substring, "was added", self.counter)
        return True

    def hasWord(self, substring):
        #substring is iterated over to fetch  individual char
        # if char is not there then return False, else update self, getChildNode
        #returns endToken which will be True depending on the node's endToken value
        for char in substring:
            if self.getChildNode(char) == None:
                return False
            self = self.getChildNode(char)
        return self.endToken

In [3]:
class TrieDS:
    def __init__(self):
        #initialized TrieDS class with an instance of a trie node.
        #Root nodes are empty and the first node of the trie structure.
        print("I am the Root Node, I have no name")
        self.RootNode = TrieNode("")  #create an instance node, but root has no character!

    def addWord(self, word, frequency):
        #word is a string to be added to current trie node instance and frequency passes an int value
        return self.RootNode.addEntry(word, frequency)

    def hasWord(self, word):
        #word is a string stored in trie
        #returns True if word is in trie, otherwise False
        #print(f"Checking {word} - {self.RootNode.hasWord(word)}")
        return self.RootNode.hasWord(word)

    def getFrequency(self, word):
        #here word is a string with corresponding frequency value 
        #Subtrings have frequencies as well, regardless of the boolean value of hasWord
        #returns an int value in the string's frequency
        return self.RootNode.getFrequency(word)

In [4]:
#created global variables outside of the functions/classes
suffixes_r = dict() 
prefixes_r = dict() 

#function to slice strings to check boundary and perform boundary testing
def getaffix(trieds, word):
    #trieds populates the TrieDS object
    #this function populates global variable dictionaries
    for i, char in enumerate(word):
        if i < (len(word) - 1):
            stem = prefix = word[:i+1] # incase of reporters this would be 'r', 're'
            suffix = end = word[i+1:] # incase of reporters this would be 'eporters', 'porters'
            
            #for suffixes
            suffixes_r.setdefault(suffix, 0)  # default to 0 to avoid overwriting existing affixes,, gives werid error i can't figure out
            if not trieds.hasWord(stem):  # Test1 to see if stem exists
                suffixes_r[suffix] -= 1 #fails then -1
            else:
                suffix_testing = score_Suffix(stem, suffix, trieds) #if it exists then initializing score_Suffix method
                suffixes_r[suffix] += suffix_testing.boundaryTests() #applying the remaining 2 tests frm within score_Suffix

            # for prefixes
            prefixes_r.setdefault(prefix, 0)
            if not trieds.hasWord(end):
                prefixes_r[prefix] -= 1
            else:
                prefix_testing = score_Prefix(prefix, end, trieds) #if it exists then initializing score_Prefix method
                prefixes_r[prefix] += prefix_testing.boundaryTests() #applying the remaining 2 tests

In [5]:
class score_Suffix:
    def __init__(self, stem, suffix, trie_ds):
        #class for scoring suffixes initialized with a stem (alphaA), a suffix (Bbeta)
        self.T = trie_ds #trie_ds is the TrieDS instance that is already populated
        self.stem = stem
        self.suffix = suffix

    def getfreq(self):
        #Builds alpha, alphaA and B frequency from stem and suffix
        #returns ints. The frequencies corresponding to strings, alpha, alphaA, and alphaAB.
        alphafreq = self.T.getFrequency(self.stem[:-1]) #apple = app--> stem[:-1]
        alphaA_freq = self.T.getFrequency(self.stem) #stem is alphaA
        Bfreq = self.T.getFrequency(self.stem + self.suffix[0]) #stem-->(alphaA) + [0] of suffix
        return alphaA_freq, alphafreq, Bfreq

# methods for testing
# test2: probability has to be approx same
# test3: probability has to be less than 1
    def boundaryTest_2(self, freq1, freq2):
        #returns True if resulting conditional probability is between 0.9 and 1
        if freq2 == 0:
            return -1
        if 0.9 <= freq1 / freq2 <= 1:
            return True

    def boundaryTest_3(self, freq1, freq2):
        #returns: True if resulting conditional probability is less than 1
        if freq2 == 0:
            return -1
        if 0 <= freq1 / freq2 < 1:
            return True

    def boundaryTests(self):
        #Calls self.getfreq to input ints to the boundary tests and change initialized score value
        #returns the suffix score (int)
        #frequencies[0]: alphaA freq value, frequencies[1]: alpha freq value, frequencies[2]: alphaAB freq value
        frequencies = self.getfreq()
        score = 0
        # performs test and +19 if it passes -1 if it fails
        if self.boundaryTest_2(frequencies[0], frequencies[1]) and self.boundaryTest_3(frequencies[2], frequencies[0]):
            score += 19
        else:
            score = -1
        return score

In [6]:
class score_Prefix(score_Suffix):

    def __init__(self, prefix, stem, trie_ds):
        #class for scoring prefixes initialized with a prefix (alphaA), a stem (Bbeta)
        # get methods from score_Suffix, frequency values are from different sliced strings
        self.T = trie_ds
        self.prefix = prefix
        self.stem = stem

    def getfreq(self):
        a_freq = self.T.getFrequency(self.prefix) #not sure? it would be prefix aka stem
        beta_freq = self.T.getFrequency(self.prefix + self.stem)
        b_freq = self.T.getFrequency(self.prefix + self.stem[0])
        return beta_freq, b_freq, a_freq

    def boundaryTests(self):
        #same as above, just the frequencies being fed are different
        frequencies = self.getfreq()
        score = 0
        if self.boundaryTest_2(frequencies[0], frequencies[1]) and self.boundaryTest_3(frequencies[0], frequencies[2]):
            score += 19
        else:
            score = -1
        return score

In [7]:
def sort_byvalue(d):
    #d is the suffixes_r, prefixes_r
    #returns a sorted affixes in descending order
    return sorted(d.items(), key = lambda affix: affix[1], reverse = True)

def rankwords(d):
    #returns a list where negative values have been removed and the affixes have been sorted by the sort_byvalue function
    return sort_byvalue(dict((i, j) for i, j in d.items() if j >= 0))
    
def topwords(d, percentage):
    #percentage can be aything between 0 to 1 so, 10% is 0.1, etc.
    #returns top affixes based on given percentage
    return d[:round(percentage * len(d))]

In [8]:
# loading english_corpus txt file
eng_df =  pd.read_csv("english_corpus.txt", delimiter='\t')
eng_df.dropna(inplace=True)

# only using words with freq > 10000 because it yields better results, got rid of one letter words and https links,
#could've used a method to filter data better?? maybe later?
#print(len(eng_df))
eng_df = eng_df[eng_df["Freq"] > 10000]
eng_df.loc[:, "word"] = eng_df["word"].str.lower() #everything to lowercase
eng_df.drop_duplicates(subset=["word"], keep="first", inplace=True) #drop duplicates
#checking for duplicate words, since before there was 'the' and 'The' twice and so on with other words!
print(eng_df.head(10))

    word      Freq
0    the  70922517
1     of  40947787
2    and  39196587
3     to  36179805
4      a  26456052
5     in  23266319
6     is  15205138
7    for  13419393
8   that  11574871
10    on  10124216


In [9]:
MyTrie = TrieDS()

eng_df.apply(lambda row: MyTrie.addWord(row["word"], row["Freq"]), axis=1) #applying addWord/ adding words to Trie
eng_df.apply(lambda row: MyTrie.hasWord(row["word"]), axis=1) #checking words on Trie
eng_df.apply(lambda row: getaffix(MyTrie, row["word"]), axis=1)


#applying rankwords and topwords method to present top 10 % of leading affixes
top_suffixes = topwords(rankwords(suffixes_r), 0.1)
top_prefixes = topwords(rankwords(prefixes_r), 0.1)

I am the Root Node, I have no name


In [10]:
print(len(top_suffixes))
print('Top suffixes:', top_suffixes[:10])

print()

print(len(top_prefixes))
print('Top prefixes:', top_prefixes[:10])

729
Top suffixes: [('s', 14326), ('ed', 3480), ('ing', 2832), ('ly', 2270), ('d', 698), ('er', 592), ('al', 562), ('.', 400), ('ive', 384), ('ion', 354)]

329
Top prefixes: [('hi', 123), ('the', 113), ('ti', 113), ('back', 112), ('no', 111), ('bi', 107), ('dea', 106), ('out', 104), ('sa', 102), ('every', 95)]


# -----------------------------------------------------------------------------------------

# Documentation

### Assignment Discussion

The above code consists of:

#### ClassMethods:
1. TrieNode class:
    - Methods:
        - getNodeChar():
            - simply returns its name character
        - getFrequency():
            - accepts a sliced or complete word (substring) and returns an int value which is the substring's frequency
        - getChildNode():
            - searches through ChildNodes{}, if node with character 'childChar' is found then returns childNode
        - addEntry():
            - Loops through each character in the word(substring)
            - Check if there is no child containing the character, create a new child for the current node and store in ChildNode
            - adding words was here I got stuck the most in the beginning because I failed to realize that self needed to be updated within the loop. In this case, failing to update self(getChildNode) would result in empty outputs.    
        - hasWord():
            - substring is iterated over to fetch  individual char
            - if char is not there then return False, else update self, getChildNode
            - returns endToken which will be True depending on the node's endToken value
            - has word was similar logic to addEntry so I did not have much problems with it


2. TrieDS class: Experimented the TrieDS with goodwords and badwords, at first I was having trouble with confirming if hasWord was working since the method kept checking the words but would return false even though the word existed. I later figured out that the self was not being updated with every iteration and thus the method couldn't actually confirm the words.
    - Methods:
        - addWord():
            - word is added to the trie node instance and frequency passes an int value
        - hasWord():
            - checks if word is in Trie and returns True or False accordingly
        - getFrequency():
            - gets word (either sliced or complete) with it's corresponding frequency value


3. getaffix class:
    - I first experimented on how to slice the strings in order to get the alpha, alphaA, alphaAB, etc. Tried using a brute force method such as:
 
                - for i, char in enumerate(substring):
                    if i < (len(substring) - 1):
                        alpha = substring[:i]
                         A = substring[i]
                         B = substring[i+1]
                         beta = substring[i+2:]
                         stem = alpha + A
                         suffix = B + beta
                 - for i, char in enumerate(words):
                        if i < (len(words) - 1):
                        stem = prefix = words[:i+1]
                        suffix = end = words[i+1:]
                        # print('prefix')
                        # print(prefix)
                        # print('suffix')
                        # print(suffix)
                        prefix1 = stem[:-1]
                        # print('new')
                        # print(prefix1)
                        B = stem + suffix[0]??
                        print('Bbeta is:')
                        print(B)??
    - Used this logic to create the class and method and then performed Test 1 on both suffixes and prefixes.
    - Test 1 is simply checking if alphaA, which is stem in case of suffix and end in case of prefix, is present using hasWord. If it passes the test then we apply the respective score_Suffix or score_Prefix method to perform the remaining tests. If it fails then -1. The prefixes and suffixes are loaded into the respective dictionaries.


4. score_Suffix class:
    - Methods:
        - getfreq():
            - Builds alpha, alphaA and B frequency from stem and suffix, and returns their corresponding int. value
            - To do this the getFrequency method from TrieDS is applied to the stem and suffix
        - boundaryTest_2():
            - A conditionalProbability is applied to test whether the probability id between 0.9 and 1 since Test 2 requires for the the frequencies to be approximately same.
            - The condition: if freq2 == 0: return -1 is applied since divions cannot happen with denominator as 0. If not for the condition then method would output ZeroDivisionError: division by zero.
        - boundaryTest_3():
            - A conditionalProbability is applied to test whether the probability id between 0 and 1, since it can't be negative. Test 3 requires for the the probability of frequencies to be less than 1.
            - - The condition: if freq2 == 0: return -1 is applied since divions cannot happen with denominator as 0. If not for the condition then method would output ZeroDivisionError: division by zero.
        - boundaryTests():
            - This method simply gets the frequencies from the getfreq method and then applies boundaryTest_2 and boundaryTest_3 to ge the scores.


5. score_Prefix class: this class got methods from score_Suffix, and is alsmost the same as score_Suffix except that we are working with different set of sliced strings and frequencies.
    - Methods:
         - getfreq():
             - This methods is the same as above except the frequency are pulled from different sliced strings: prefix (alphaA), a stem (Bbeta)
         - boundaryTests():
             - This method is the same as the one in score_Suffix but with different frequencies.

#### Functions:

6. sort_byvalue:
    - This function sorts the affixes in descending order using the sorted() method.
7. rankwords: 
    - This function removes affixes with negative values and applies the sort_byvalue function.
8. topwords:
    - This function returns top affixes based on given percentage. It is a simple x% of len(d) function.
    
#### Data preprocessing:

I had some problems reading the data in the beginning as the text file has some lines that did not work well with pandas implementation. So, I look at the text file and removed the '---' lines as well as the first few note lines so that i could read the file using pandas and get a simple dataframe. Then, upon inspection I saw that some of the words weren't really words but links and that some were repeated with upper and lower case letters. So, I performed some cleaning, changed all words to lowercase, filtered out words that had frequency lower than 10,000 and got rid of duplicates.
    

### Observations:

I applied the topwords method to the ranked suffixes and prefixes, to get the top 10% of the affixes. I got the following results:

length of the top 10% of suffixes: 729

Top suffixes[:10]: [('s', 14326), ('ed', 3480), ('ing', 2832), ('ly', 2270), ('d', 698), ('er', 592), ('al', 562), ('.', 400), ('ive', 384), ('ion', 354)]


length of the top 10% of prefixes: 329

Top prefixes[:10]: [('hi', 123), ('the', 113), ('ti', 113), ('back', 112), ('no', 111), ('bi', 107), ('dea', 106), ('out', 104), ('sa', 102), ('every', 95)]

The suffixes seem to make sense and the fact that 'ing', 'ed', 'ly' are listed among the top tells me that the suffixes are computed well. The prefixes such as 'bi', 'out', 'every' were also good outcomes as they were listed high among the ranked prefixes.

### Segmentation:

I experimented with various methods for segmentation but struggled with 'TypeErrors', and couldn't figure out how to solve them correctly. The logic I tried to implement for segmentation was:
- Loop over suffix from top_suffixes
- Check if any of the stored words ends with the suffix
- If yes then store them in the new dictionary
- The use another loop to only fetch the most frequent words from within the dictionary

This was the logic I intended to apply, I'm not sure if it would have worked but it seemed like the most simple way to implement segmentation.

### References
- https://albertauyeung.github.io/2020/06/15/python-trie.html/
- https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/22453/Lushtak_washington_0250O_11149.pdf?sequence=1
- https://www.aleksandrhovhannisyan.com/blog/trie-data-structure-implementation-in-python/
- https://slideplayer.com/slide/5016724/